# Attempt 3

Here is an outline of what has been accomplished and modified

In [22]:
from IPython.display import Image
from IPython.core.display import HTML

## Purpose

In ASP, the multi-agent pathfinding (MAPF) problem is very well explored.  Documented are several methods for solving the problem efficiently.  However, many of these are designed for four- or eight-connected cells.  What separates the Flatland environment from traditional MAPF environments is its variation of possible connections and transitions between cells, which are not uniform throughout.

For this reason, the first major step in building out an encoding is finding the most efficient way to build out a path for a single agent.

## Overview

Each problem then contains three separate files:
1. `types.lp` which defines the track types and possible transitions, and is consistent across all instances
2. `environment.lp` which is the custom environment that is being tested
    1. This defines each cell location and its track type/rotation
    2. It also contains `start()` and `end()` locations for agents, as well as a `maxTime()` parameter
3. `encoding.lp` which is consistent across all instances


---

## `types.lp`
> Cells are represented differently

* Originally, a hypergraph (specific to each environment) was established to define transitions between cells
* Then, cells were defined by a matrix
* Now, cells are defined by a set of possible moves

### Hypergraph

`path( cell(0,0), cell(0,1), cell(0,2) )` <br>
`path( cell(0,2), cell(0,1), cell(0,0) )`

Two primary problems:
1. This could not be reused, as it was specific to each environment.
2. The checking process was extremely slow.


### Matrix

`Type 1/90`<br>
`• L F R` <br>
`N 0 1 0` <br>
`W 0 0 0` <br>
`S 0 1 0` <br>
`E 0 0 0` <br>

This works, but:
1. It requires Python.
2. It is difficult to represent in ASP, at least in a readable way.

### Sets of moves

`Type 1/90 {(North,Forward),(South,Forward)}`

In [28]:
Image(url= "type_01.png", width = 300)

`type( (Track Type, Rotation), (Origin, Direction) )`

In [31]:
%%clingo 0

% Track Type 1 - Straight
type((1,0) , (n,f)).    type((1,0) , (s,f)).
type((1,90), (w,f)).    type((1,90), (e,f)).

clingo version 5.6.2
Reading from stdin
Solving...
Answer: 1
type((1,0),(n,f)) type((1,0),(s,f)) type((1,90),(w,f)) type((1,90),(e,f))
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.003s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.001s


---

## `environment.lp`
Each cell and track type is defined.  Below are the environment parameters.

In [ ]:
# %%clingo 0

% Potsdam environment
cell((0,5), (7,90)).
cell((1,5), (1,90)).
cell((2,5), (1,90)).
cell((3,5), (22,90)).
cell((3,6), (1,0)).
[...]

% parameters
start(cell(24,4), dir(w)).
end(cell(3,8)).
maxTime(25).

---

## `encoding.lp`
The encoding has been adjusted to account for the new cell representations.

In [ ]:
%%clingo 0

% determine first two cells
at(cell(X0,Y0), 0, D) :- start(cell(X0,Y0), dir(D)).
at(cell(X1,Y1), 1, DD) :- at(cell(X0,Y0), 0, D), swap(D,From), offset((From,f), (DX,DY), DD), X1=X0+DX, Y1=Y0+DY.

% determine all subsequent moves
{ do(A,T) : move(A) } = 1 :- maxTime(M), T=2..M.
at(cell(X+DX,Y+DY), T, DD) :-  
    do(A,T), at(cell(X,Y), T-1, D), swap(D,From), cell((X,Y), (Type,Rotation)), 
    offset((From,A), (DX,DY), DD), type((Type,Rotation), (From,A)).

% must reach the goal
:- end(cell(X,Y)), not at(cell(X,Y), _, _).

% must be a valid cell
:- at(cell(X,Y), T, D), not cell((X,Y), _).

#show at/3.

### Example

In [23]:
Image(url= "potsdam_example.png")

**Previously, a path of 16 steps was taking about 8 minutes to calculate.  Here, a path of 25 steps takes 0.008 seconds.**
```
% parameters
start(cell(24,4), dir(w)).
end(cell(3,8)).
maxTime(25).
```
---
```
Reading from encoding.lp ...
Solving...
Answer: 1
at(cell(24,4),0,w) at(cell(23,4),1,w) at(cell(23,5),2,n) at(cell(22,5),3,w) at(cell(21,5),4,w) at(cell(20,5),5,w) at(cell(19,5),6,w) at(cell(18,5),7,w) at(cell(17,5),8,w) at(cell(16,5),9,w) at(cell(15,5),10,w) at(cell(14,5),11,w) at(cell(13,5),12,w) at(cell(12,5),13,w) at(cell(11,5),14,w) at(cell(10,5),15,w) at(cell(9,5),16,w) at(cell(8,5),17,w) at(cell(7,5),18,w) at(cell(6,5),19,w) at(cell(5,5),20,w) at(cell(4,5),21,w) at(cell(3,5),22,w) at(cell(3,6),23,n) at(cell(3,7),24,n) at(cell(3,8),25,n)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.008s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.008s
```

---

## Next steps

* Including multiple agents
* Working with Flatland
    * Visualizing the results
    * Generating new environments

In [20]:
Image(url= "track_types_id.png")

A silly custom example.

In [30]:
Image(url= "silly.png", width=750)